In [1]:
try:
    import pygraphblas
except:
    !mamba install -c conda-forge pygraphblas

In [16]:
from pygraphblas import *
import numpy as np

In [3]:
def cohen(A, U, L): return L.mxm(U, mask=A)

In [4]:
prob = dict(Matrix.ssget('Newman/karate'))
Mat = prob['karate.mtx'].cast(INT64)

In [5]:
A = Mat.triu()
B = Mat.tril()
M = Mat
C = cohen(Mat, A, B)

In [6]:
A_sp = A.to_scipy_sparse()
B_sp = B.to_scipy_sparse()
M_sp = M.to_scipy_sparse()
C_sp = C.to_scipy_sparse()

In [31]:
def np_to_str(npa):
    return np.array2string(npa, separator=",").replace("[", "{").replace("]", "}")

In [32]:
data = (np_to_str(A_sp.indptr), np_to_str(A_sp.indices), np_to_str(A_sp.data), 
 np_to_str(B_sp.indptr), np_to_str(B_sp.indices), np_to_str(B_sp.data),
 np_to_str(M_sp.indptr), np_to_str(M_sp.indices), np_to_str(M_sp.data),
 np_to_str(C_sp.indptr), np_to_str(C_sp.indices), np_to_str(C_sp.data))

output_str = """
#include <vector>
#include <cstdint>

template<typename T_A, typename T_B, typename T_C, typename T_M>
class TestData {

public:
    TestData(  std::vector<std::int64_t> A_indptr_,
    std::vector<std::int64_t> A_indices_,
    std::vector<T_A> A_data_,

    std::vector<std::int64_t> B_indptr_,
    std::vector<std::int64_t> B_indices_,
    std::vector<T_B> B_data_,


    std::vector<std::int64_t> C_indptr_,
    std::vector<std::int64_t> C_indices_,
    std::vector<T_C> C_data_,

    std::vector<std::int64_t> M_indptr_,
    std::vector<std::int64_t> M_indices_,
    std::vector<T_M> M_data_):
        A_indptr(A_indptr_), A_indices(A_indices_), A_data(A_data_),
        B_indptr(B_indptr_), B_indices(B_indices_), B_data(B_data_),
        C_indptr(C_indptr_), C_indices(C_indices_), C_data(C_data_),
        M_indptr(M_indptr_), M_indices(M_indices_), M_data(M_data_){}


  std::vector<std::int64_t> A_indptr;
  std::vector<std::int64_t> A_indices;
  std::vector<T_A> A_data;
  
  std::vector<std::int64_t> B_indptr;
  std::vector<std::int64_t> B_indices;
  std::vector<T_B> B_data;
  
  
  std::vector<std::int64_t> C_indptr;
  std::vector<std::int64_t> C_indices;
  std::vector<T_C> C_data;

  std::vector<std::int64_t> M_indptr;
  std::vector<std::int64_t> M_indices;
  std::vector<T_M> M_data;

};

template<typename T_A, typename T_B, typename T_C, typename T_M>
std::unique_ptr<TestData<T_A, T_B, T_C, T_M>> make_karate_tricount() {

    std::vector<std::int64_t> A_indptr = %s;
    std::vector<std::int64_t> A_indices = %s;
    std::vector<T_A> A_data = %s;

    std::vector<std::int64_t> B_indptr = %s;
    std::vector<std::int64_t> B_indices = %s;
    std::vector<T_B> B_data = %s;

    std::vector<std::int64_t> M_indptr = %s;
    std::vector<std::int64_t> M_indices = %s;
    std::vector<T_M> M_data = %s;

    std::vector<std::int64_t> C_indptr = %s;
    std::vector<std::int64_t> C_indices = %s;
    std::vector<T_C> C_data = %s;

    TestData<T_A, T_B, T_C, T_M> karate_tricount(A_indptr, A_indices, A_data,
                                                 B_indptr, B_indices, B_data,
                                                 C_indptr, C_indices, C_data,
                                                 M_indptr, M_indices, M_data);

    return std::make_unique<TestData<T_A, T_B, T_C, T_M>>(karate_tricount);
}



TestData karate_tricount;
karate.A_indptr = %s;
karate.A_indices = %s;
karate.A_data = %s;

karate.B_indptr = %s;
karate.B_indices = %s;
karate.B_data = %s;

karate.M_indptr = %s;
karate.M_indices = %s;
karate.M_data = %s;

karate.C_indptr = %s;
karate.C_indices = %s;
karate.C_data = %s;
""" % data

In [33]:
output_str

'\ntemplate<typename T_A, typename T_B, typename T_C, typename T_M>\nstruct TestData {\n\n  std::vector<int64_t> A_indptr;\n  std::vector<int64_t> A_indices;\n  std::vector<T_A> A_data;\n  \n  std::vector<int64_t> B_indptr;\n  std::vector<int64_t> B_indices;\n  std::vector<T_B> B_data;\n  \n  \n  std::vector<int64_t> C_indptr;\n  std::vector<int64_t> C_indices;\n  std::vector<T_C> C_data;\n\n  std::vector<int64_t> M_indptr;\n  std::vector<int64_t> M_indices;\n  std::vector<T_M> M_data;\n\n}\n\n\nTestData karate_tricount;\nkarate.A_indptr = { 0,16,24,32,35,37,40,41,41,44,45,45,45,45,46,48,50,50,50,52,53,55,55,57,\n 62,65,66,68,69,71,73,75,77,78,78};\nkarate.A_indices = { 1, 2, 3, 4, 5, 6, 7, 8,10,11,12,13,17,19,21,31, 2, 3, 7,13,17,19,21,30,\n  3, 7, 8, 9,13,27,28,32, 7,12,13, 6,10, 6,10,16,16,30,32,33,33,33,32,33,\n 32,33,32,33,33,32,33,32,33,25,27,29,32,33,25,27,31,31,29,33,33,31,33,32,\n 33,32,33,32,33,33};\nkarate.A_data = {1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1

In [34]:
def store_file(output_string, filename = "test_data.hpp"):
    with open(filename, 'w') as f:
        f.write(output_string)

In [35]:
store_file(output_str)